In [1]:
import sys
sys.path.insert(0, ".")


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import clock

clock.weekly_job_1()

Fitting 36 kabko with [] test_splits and 1 process
Fitting KAB. BANGKALAN with [] test_splits
Done fitting KAB. BANGKALAN with [] test_splits
Fitting KAB. BANYUWANGI with [] test_splits
Done fitting KAB. BANYUWANGI with [] test_splits
Fitting KAB. BLITAR with [] test_splits
Done fitting KAB. BLITAR with [] test_splits
Fitting KAB. BOJONEGORO with [] test_splits
Done fitting KAB. BOJONEGORO with [] test_splits
Fitting KAB. BONDOWOSO with [] test_splits
Done fitting KAB. BONDOWOSO with [] test_splits
Fitting KAB. GRESIK with [] test_splits
Done fitting KAB. GRESIK with [] test_splits
Fitting KAB. JEMBER with [] test_splits
Done fitting KAB. JEMBER with [] test_splits
Fitting KAB. JOMBANG with [] test_splits
Done fitting KAB. JOMBANG with [] test_splits
Fitting KAB. KEDIRI with [] test_splits
Done fitting KAB. KEDIRI with [] test_splits
Fitting KAB. LAMONGAN with [] test_splits
Done fitting KAB. LAMONGAN with [] test_splits
Fitting KAB. LUMAJANG with [] test_splits
Done fitting KAB. LUMAJ

Traceback (most recent call last):
  File ".\clock.py", line 33, in weekly_job_1
  File ".\main.py", line 40, in fit
    fitting.fit_all(test_splits)
  File ".\core\fitting.py", line 51, in fit_all
    print("Fitting %s with %s test_splits, out of %s kabko" % (selected_kabko, str(test_splits), str(count)))
  File ".\core\fitting.py", line 34, in fit
    with db.get_conn() as conn, conn.cursor() as cur:
  File "C:\ProgramData\Anaconda3\lib\contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "c:\users\lenovox230i\github\prediksicovidjatim-core\prediksicovidjatim\database.py", line 29, in get_conn
    conn = self.conn_pool.getconn(key)
  File "C:\ProgramData\Anaconda3\lib\site-packages\psycopg2\pool.py", line 169, in getconn
    return self._getconn(key)
  File "C:\ProgramData\Anaconda3\lib\site-packages\psycopg2\pool.py", line 93, in _getconn
    return self._connect(key)
  File "C:\ProgramData\Anaconda3\lib\site-packages\psycopg2\pool.py", line 63, in _connect
    co

In [4]:
import os
os.getenv("ASSDASD")

In [6]:
import datetime
datetime.datetime.now()

datetime.datetime(2020, 7, 29, 0, 32, 37, 827563)

In [7]:
undefined

NameError: name 'undefined' is not defined